# Granules

In [1]:
import starepandas
import geopandas
import pystare
import matplotlib.pyplot as plt
import importlib
import copy

In [2]:
fname = '../tests/data/granules/MOD05_L2.A2019336.0000.061.2019336211522.hdf'
modis = starepandas.read_granule(fname, latlon=True, sidecar=True, xy=True)

In [ ]:
modis = copy.copy(modis[(modis.y>250) & (modis.x<150)])

In [ ]:
modis

In [ ]:
trixels = modis.make_trixels()
modis.set_trixels(trixels, inplace=True)

In [ ]:
geom = geopandas.points_from_xy(modis.lon, modis.lat)
modis.set_geometry(geom, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10), dpi=100)
ax.grid(True)

modis.plot(trixels=False, ax=ax, marker='.', markersize=20)
modis.plot(trixels=True, color='r', ax=ax, lw=0.5)

#plt.savefig('modis.png')

## Plotting footprints

In [ ]:
fname = '../tests/data/granules/MOD05_L2.A2019336.0000.061.2019336211522.hdf'
mod05 = starepandas.io.granules.Mod05(fname)

In [ ]:
mod05.read_sidecar_cover()
mod05.stare_cover

## Manually getting the 2D STARE array

In [ ]:
import netCDF4
from pyhdf.SD import SD
import numpy
import pystare
import datetime

In [ ]:
hdf = SD(fname)
lon = hdf.select('Longitude').get().astype(numpy.double)
lat = hdf.select('Latitude').get().astype(numpy.double)

In [ ]:
start = datetime.datetime.now()
sids = pystare.from_latlon_2d(lat=lat, lon=lon, adapt_level=True)
datetime.datetime.now()-start

In [ ]:
print(pystare.spatial_resolution(sids).min())
print(pystare.spatial_resolution(sids).max())

## Intersecting Data 

In [ ]:
world = geopandas.read_file(geopandas.datasets.get_path('naturalearth_lowres'))
world.sort_values(by='name', inplace=True)
world = starepandas.STAREDataFrame(world)
sids = world.make_sids(level=6)
world.set_sids(sids, inplace=True)
trixels = world.make_trixels()
world.set_trixels(trixels, inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(4,4), dpi=200)
ax.grid(True)

country = world[world.name=='Iceland']
country.plot(ax=ax, trixels=True, boundary=True, color='y', zorder=1)
country.plot(ax=ax, trixels=False, facecolor="none", edgecolor='blue', zorder=1)
modis.plot(ax=ax, color='red', trixels=False, zorder=0, linewidth=0.1)

In [ ]:
a = modis.stare_intersects(country['sids'].iloc[0])
a.any()

In [18]:
len(country['sids'].iloc[0])

17